# Практика 8: Методы защиты от атак на модели ИИ
Картунчиков Артем ББМО-01-23

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


/Users/a.kartunchikov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Загрузка данных MNIST

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

train_images_1 = train_images[:1000]
train_labels_1 = train_labels[:1000]


# 1. Защита с помощью Adversarial Training

## Функция для FGSM атаки

In [3]:
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

## Функция для генерации противоречивых примеров

In [4]:
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))

    return np.array(adversarial_images)


## Создание модели

In [5]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Обучение модели с противоречивыми примерами

In [6]:
def adversarial_training(model, train_image_1, train_labels, epsilon):
    for epoch in range(10):
        for i in range(0, len(train_images_1), 32):
            batch_images = train_images_1[i:i+32]
            batch_labels = train_labels_1[i:i+32]
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            model.train_on_batch(combined_images, combined_labels)

## Инициализация модели

In [7]:
model = create_model()

/Users/a.kartunchikov/Library/Python/3.9/lib/python/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Тренировка модели с защитой на противоречивых примерах

In [8]:
adversarial_training(model, train_images_1, train_labels_1, epsilon=0.1)

## Сохранение защищенной модели

In [9]:
model.save('adversarial_trained_model.h5')

# 2. Градиентная маскировка (Gradient Masking)

## Обновление модели для градиентной маскировки

In [10]:
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


## Обучение модели с градиентной маскировкой

In [11]:
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 531us/step - accuracy: 0.8707 - loss: 0.4687
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 526us/step - accuracy: 0.9643 - loss: 0.1174
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 547us/step - accuracy: 0.9758 - loss: 0.0790
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 533us/step - accuracy: 0.9814 - loss: 0.0601
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 535us/step - accuracy: 0.9867 - loss: 0.0452


## Сохранение модели с градиентной маскировкой

In [12]:
masked_model.save('masked_model.h5')

## 3. Регуляризация и нормализация для повышения устойчивости

## Модель с регуляризацией и нормализацией

In [13]:
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model

## Обучение модели с регуляризацией и нормализацией

In [14]:
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 607us/step - accuracy: 0.7942 - loss: 1.3296
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 606us/step - accuracy: 0.8703 - loss: 0.6272
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 610us/step - accuracy: 0.8671 - loss: 0.6168
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 603us/step - accuracy: 0.8665 - loss: 0.6179
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 602us/step - accuracy: 0.8611 - loss: 0.6197


## Сохранение модели с регуляризацией

In [15]:
regularized_model.save('regularized_model.h5')

# 4. Оценка моделей на противоречивых примерах

## Adversarial Training

In [16]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model,
test_images, test_labels, epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 0.5653 - loss: 1.3008
Accuracy of protected model on adversarial examples: 0.6190000176429749


## regularized_model

In [17]:

# Загрузка атакованной модели
regularized_model = tf.keras.models.load_model('regularized_model.h5')
masked_model = tf.keras.models.load_model('masked_model.h5')

# Оценка модели с регуляризацией и нормализацией на противоречивых примерах
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

# Оценка модели с градиентной маскировкой на противоречивых примерах
test_loss, test_acc = masked_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - accuracy: 0.6585 - loss: 1.1872
Accuracy of regularized model on adversarial examples: 0.7013999819755554
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 280us/step - accuracy: 0.7436 - loss: 0.8522
Accuracy of masked model on adversarial examples: 0.7889000177383423


# Выводы

Тестирование на противоречивых примерах показало, что различные методы защиты улучшают устойчивость модели к атакам.

Adversarial Training продемонстрировал точность 61.9% на противоречивых примерах, что выше по сравнению с исходной моделью.

Модель с регуляризацией достигла точности 70.1%, что указывает на эффективность регуляризации как метода защиты.

Маскирующая модель оказалась наиболее устойчивой, показав точность 78.9% на противоречивых примерах.

Различные методы защиты, такие как adversarial training, регуляризация и маскирование, повышают устойчивость модели к атакам, причем маскирование оказалось самым эффективным методом защиты в данной работе.